# Unit Testing `GiRaFFE_NRPy`: $A_k$ to $B^i$

### Author: Patrick Nelson

This notebook validates our A-to-B solver for use in `GiRaFFE_NRPy`. Because the original `GiRaFFE` used staggered grids and we do not, we can not trivially do a direct comparison to the old code. Instead, we will compare the numerical results with the expected analytic results. 

**Module Status:** <font color=red><b> In-Progress </b></font>

**Validation Notes:** This module will validate the routines in [Tutorial-GiRaFFE_HO_C_code_library-A2B](../Tutorial-GiRaFFE_HO_C_code_library-A2B.ipynb).

It is, in general, good coding practice to unit test functions individually to verify that they produce the expected and intended output. Here, we expect our functions to produce the correct cross product in an arbitrary spacetime. To that end, we will choose functions that are easy to differentiate, but lack the symmetries that would trivialize the finite-difference algorithm. Higher-order polynomials are one such type of function. 

We will start with the simplest case - testing the second-order solver. In second-order finite-differencing, we use a three-point stencil that can exactly differentiate polynomials up to quadratic. So, we will use cubic functions three variables. For instance,

\begin{align}
A_x &= ax^3 + by^3 + cz^3 + dy^2 + eyz + fz^2 + g \\
A_y &= hx^3 + ly^3 + mz^3 + nx^2 + oxz + pz^2 + q \\
A_z &= rx^3 + sy^3 + tz^3 + ux^2 + vxy + wy^2 + \alpha. \\
\end{align}

It will be much simpler to let NRPy+ handle most of this work. So, we will import the core functionality of NRPy+, build the expressions, and then output them using `outputC()`.

In [1]:
import shutil, os, sys           # Standard Python modules for multiplatform OS-level functions
# First, we'll add the parent directory to the list of directories Python will check for modules.
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

from outputC import *            # NRPy+: Core C code output module
import finite_difference as fin  # NRPy+: Finite difference C code generation module
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import grid as gri               # NRPy+: Functions having to do with numerical grids
import loop as lp                # NRPy+: Generate C code loops
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import reference_metric as rfm   # NRPy+: Reference metric support
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

thismodule = "Unit_Test_GiRaFFE_NRPy_Ccode_library_A2B"
a,b,c,d,e,f,g,h,l,m,n,o,p,q,r,s,t,u,v,w,alpha = par.Cparameters("REAL",thismodule,["a","b","c","d","e","f","g","h","l","m","n","o","p","q","r","s","t","u","v","w","alpha"],10.0)
gammadet = gri.register_gridfunctions("AUXEVOL","gammadet")

DIM = 3
par.set_parval_from_str("grid::DIM",DIM)

par.set_parval_from_str("reference_metric::CoordSystem","Cartesian")
rfm.reference_metric()
gri.register_gridfunctions_for_single_rank1
x = rfm.xxCart[0]
y = rfm.xxCart[1]
z = rfm.xxCart[2]

AD = ixp.register_gridfunctions_for_single_rank1("EVOL","AD")
AD[0] = a*x**3 + b*y**3 + c*z**3 + d*y**2 + e*y*z + f*z**2 + g
AD[1] = h*x**3 + l*y**3 + m*z**3 + n*x**2 + o*x*z + p*z**2 + q
AD[2] = r*x**3 + s*y**3 + t*z**3 + y*x**2 + v*x*y + w*y**2 + alpha


Next, we'll let NRPy+ compute derivatives analytically according to $$B^i = \frac{[ijk]}{\sqrt{\gamma}} \partial_j A_k.$$ Then we can carry out two separate tests to verify the numerical derivatives. First, we will verify that when we let the cubic terms be zero, the two calculations of $B^i$ agree to roundoff error. Second, we will verify that when we set the cubic terms, our error is dominated by trunction error that converges to zero at the expected rate. 

In [2]:
import WeylScal4NRPy.WeylScalars_Cartesian as weyl
LeviCivitaDDD = weyl.define_LeviCivitaSymbol_rank3()
LeviCivitaUUU = ixp.zerorank3()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            LeviCivitaUUU[i][j][k] = LeviCivitaDDD[i][j][k] / sp.sqrt(gammadet)
            
B_analyticU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","B_analyticU")
for i in range(DIM):
    B_analyticU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            B_analyticU[i] += LeviCivitaUUU[i][j][k] * sp.diff(AD[k],rfm.xxCart[j])


Now that we have our vector potential and analytic magnetic field to compare against, we will start writing our unit test. For this test, we cannot use only point; we will need a small grid. As we are testing a three-point stencil, we can get away with a minimal $3 \times 3$ grid. We'll also import common C functionality and set the standard macros for NRPy+ style memory access.

In [3]:
out_string = """
// These are common packages that we are likely to need.
#include "stdio.h"
#include "stdlib.h"
#include "math.h"
#include "string.h" // Needed for strncmp, etc.
#include "stdint.h" // Needed for Windows GCC 6.x compatibility
#include <time.h>   // Needed to set a random seed.

// Standard GRFFE parameters:
const int Nxx_plus_2NGHOSTS[3] = {3,3,3};

// Standard NRPy+ memory access:
#define IDX4(g,i,j,k) \
( (i) + Nxx_plus_2NGHOSTS[0] * ( (j) + Nxx_plus_2NGHOSTS[1] * ( (k) + Nxx_plus_2NGHOSTS[2] * (g) ) ) )
#define IDX3(i,j,k) ( (i) + Nxx_plus_2NGHOSTS[0] * ( (j) + Nxx_plus_2NGHOSTS[1] * (k) ) )
// Assuming idx = IDX3(i,j,k). Much faster if idx can be reused over and over:
#define IDX4pt(g,idx)   ( (idx) + (Nxx_plus_2NGHOSTS[0]*Nxx_plus_2NGHOSTS[1]*Nxx_plus_2NGHOSTS[2]) * (g) )

"""

We'll now define the gridfunction names.

In [4]:
out_string += """
// Let's also #define the NRPy+ gridfunctions
#define GAMMADETGF 0
#define AD0GF 1
#define AD0GF 2
#define AD0GF 3
#define B_analyticU0GF 4
#define B_analyticU1GF 5
#define B_analyticU2GF 6
#define B_numericU0GF 7
#define B_numericU1GF 8
#define B_numericU2GF 9
#define NUM_AUXEVOL_GFS 10

"""

Now, we'll handle the different A2B codes. There are several things to do here. First, we'll add `#include`s to the C code so that we have access to the functions we want to test. Relative to this tutorial, they are in the subfolder `A2B`.

In [5]:
out_string += """
#include "A2B/driver_AtoB.c" // This file contains both functions we need.

"""

We also should write a function that will use the analytic formulae for $B^i$. Then, we'll need to call the function from the module `GiRaFFE_HO_A2B` to generate the different header files.

In [6]:
out_string += """
void calculate_exact_BU(const int i0min, const int i0max,
                        const int i1min, const int i1max,
                        const int i2min, const int i2max,
                        const double *evol_gfs, double *out_gfs) {
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
"""

B_analyticU_to_print   = [\
                           lhrh(lhs=gri.gfaccess("out_gfs","B_analyticU0"),rhs=B_analyticU[0]),\
                           lhrh(lhs=gri.gfaccess("out_gfs","B_analyticU1"),rhs=B_analyticU[1]),\
                           lhrh(lhs=gri.gfaccess("out_gfs","B_analyticU2"),rhs=B_analyticU[2]),\
                          ]
B_analyticU_kernel = fin.FD_outputC("returnstring",B_analyticU_to_print,params="outCverbose=False")
out_string += B_analyticU_kernel
print(B_analyticU_kernel)

out_string += """        
    }
}

"""

gri.glb_gridfcs_list = []
import GiRaFFE_HO.GiRaFFE_HO_A2B as A2B
# We'll generate these into the A2B subdirectory since that's where the functions
# we're testing expect them to be.
A2B.GiRaFFE_HO_A2B("A2B/")


{
   /* 
    * NRPy+ Finite Difference Code Generation, Step 1 of 1: Read from main memory and compute finite difference stencils:
    */
   const double gammadet = auxevol_gfs[IDX4(GAMMADETGF, i0,i1,i2)];
   /* 
    * NRPy+ Finite Difference Code Generation, Step 2 of 1: Evaluate SymPy expressions and write to main memory:
    */
   const double tmp0 = pow(gammadet, -1.0/2.0);
   const double tmp1 = 2*xx2;
   const double tmp2 = 3*pow(xx2, 2);
   const double tmp3 = pow(xx0, 2);
   const double tmp4 = 2*xx1;
   const double tmp5 = 3*pow(xx1, 2);
   const double tmp6 = 3*tmp3;
   auxevol_gfs[IDX4(B_ANALYTICU0GF, i0, i1, i2)] = -tmp0*(m*tmp2 + o*xx0 + p*tmp1) + tmp0*(s*tmp5 + tmp3 + tmp4*w + v*xx0);
   auxevol_gfs[IDX4(B_ANALYTICU1GF, i0, i1, i2)] = tmp0*(c*tmp2 + e*xx1 + f*tmp1) - tmp0*(r*tmp6 + tmp4*xx0 + v*xx1);
   auxevol_gfs[IDX4(B_ANALYTICU2GF, i0, i1, i2)] = -tmp0*(b*tmp5 + d*tmp4 + e*xx2) + tmp0*(h*tmp6 + 2*n*xx0 + o*xx2);
}


Wrote to file "A2B/B_from_A_order10.h"
Wrote to file

**TODO: Declare coordinates as gridfunctions**